In [1]:
pip install plaid-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
from plaid import Client
import os


clientid = os.getenv('PLAID_CLIENT_ID')
secret = os.getenv('PLAID_SECRET')


client = Client(client_id=clientid, secret=secret, environment='sandbox')

# For example, create a link token
response = client.LinkToken.create({
    'user': {'client_user_id': 'unique_user_id'},
    'client_name': 'Your App',
    'products': ['auth', 'transactions'],
    'country_codes': ['US'],
    'language': 'en',
    'redirect_uri': 'https://your-redirect-uri.com',
})
print(response)


ImportError: cannot import name 'Client' from 'plaid' (/opt/anaconda3/envs/dev/lib/python3.10/site-packages/plaid/__init__.py)